In [2]:
from manim import *
from numpy import linalg as npl
import matplotlib.pyplot as plt
import jupyter_capture_output
from scipy.interpolate import make_interp_spline, BSpline

video_scene = " -v WARNING --progress_bar None --disable_caching mfor_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s mfor_Scene"

Manim Community v0.16.0.post0

Jupyter Capture Output v0.0.8


In [33]:
# Trajektorie der Parabel: a*(x - x_shift)**2 + b
def traj_bird(t):  
    return (2.5*t - 16, 12.5*t - 2 * t**2, 0)


# Trajektorie des bewegten reference-frames: a*x + b
def traj_cat(t):
    x = -2*t
    a = -7.5/10
    return (x, a*x, 0)

In [35]:
%%capture_video --path "animations/moved_frame_of_reference/moved_frame_of_reference.mp4"
%%manim -qm --fps 20 $video_scene

class mfor_Scene(Scene):
    def construct(self):
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)

        origin = [3, -1, -1]

        text1 = Text("(a) Ruhendes Bezugssystem", font_size = 36).align_on_border(UP + LEFT, buff = 0.5)
        text2 = Text("(b) Bewegtes Bezugssystem", font_size = 36).align_on_border(UP + LEFT, buff = 0.5)

        npla = NumberPlane(
            x_range = [-50, 50, 2], y_range  = [-50, 50, 2], x_length = 50, y_length = 20, 
            axis_config = {"include_numbers": False, "stroke_opacity": 0.25}, background_line_style = {"stroke_opacity": 0.25}).move_to(origin)  

        npla2 = NumberPlane(x_range = [-50, 50, 2], y_range  = [-50, 50, 2], x_length = 50, y_length = 20).move_to(origin)       

        cat = ImageMobject("media/images/manim_animations/leothekiller_t.png").move_to(npla.c2p(traj_cat(0)[0], traj_cat(0)[1], 3))
        bird = ImageMobject("media/images/manim_animations/birdyvictim_t.png").move_to(npla.c2p(traj_bird(0)[0], traj_bird(0)[1], 1))

        cat_x_axis = Line(npla.c2p(-50, traj_cat(0)[1], 0), npla.c2p(50, traj_cat(0)[1], 0), color = WHITE, stroke_width = 0.5)
        cat_y_axis = Line(npla.c2p(traj_cat(0)[0], -50, 0), npla.c2p(traj_cat(0)[0], 50, 0), color = WHITE, stroke_width = 0.5)

        cat_x = Text("x'", font_size = 16).move_to([6.75, npla.c2p(0, traj_cat(0)[1] + 0.5, 0)[1], 0])
        cat_y = Text("y'", font_size = 16).move_to([npla.c2p(traj_cat(0)[0] + 0.5, 0, 0)[0], 3.75, 0])

        x = Text("x", font_size = 16).move_to([6.75, npla.c2p(0, traj_cat(0)[1] + 0.5, 0)[1], 0])
        y = Text("y", font_size = 16).move_to([npla.c2p(traj_cat(0)[0] + 0.5, 0, 0)[0], 3.75, 0])

        # z-Indizes
        npla.z_index = -1
        cat.z_index = 2
        bird.z_index = 4
        
        def dot_fadeout_updater(dot):
            if dot.counter > 0:
                dot.fill_opacity -= 1/dot.counter
                dot.set_opacity(dot.fill_opacity)
                dot.counter -= 1               


        def bird_updater(obj):
            t = time.get_value()
            x = traj_bird(t)[0]
            y = traj_bird(t)[1]

            dot1 = Dot(radius = 0.05, fill_color = RED, fill_opacity = 0.5).move_to(npla.c2p(x, y, -1))
            dot1.z_index = 3
            dot1.counter = 200
            dot1.fill_opacity = 1
            dot1.set_opacity(dot1.fill_opacity)
            self.add(dot1)
            dot1.add_updater(dot_fadeout_updater)

            obj.move_to(npla.c2p(x, y, 0))

        def cat_updater(obj):
            t = time.get_value()
            x = traj_cat(t)[0]
            y = traj_cat(t)[1]
            obj.move_to(npla.c2p(x, y, 0))

        def cat_x_axis_updater(axis):
            t = time.get_value()
            y = traj_cat(t)[1]
            axis.move_to(npla.c2p(0, y, 0))

        def cat_y_axis_updater(axis):
            t = time.get_value()
            x = traj_cat(t)[0]
            axis.move_to(npla.c2p(x, 0, 0))

        def cat_x_updater(label):
            t = time.get_value()
            y = npla.c2p(0, traj_cat(t)[1] + 0.5, 0)[1]
            label.move_to([6.75, y, 0])

        def cat_y_updater(label):
            t = time.get_value()
            x = npla.c2p(traj_cat(t)[0] + 0.5, 0, 0)[0]
            label.move_to([x, 3.75, 0])

        def x_updater(label):
            t = time.get_value()
            y = npla.c2p(0, traj_cat(0)[1] + 0.5, 0)[1]
            label.move_to([6.75, y, 0])

        def y_updater(label):
            t = time.get_value()
            x = npla.c2p(traj_cat(0)[0] + 0.5, 0, 0)[0]
            label.move_to([x, 3.75, 0])

        def npla_updater(plane):
            t = time.get_value()
            x = -traj_cat(t)[0]
            y = -traj_cat(t)[1]      
            plane.move_to(npla2.c2p(x, y, 0))


########## Animation ##########

        self.add(npla)
        self.play(Write(text1), run_time = 1.5)
        self.play(FadeIn(cat), FadeIn(bird), FadeIn(cat_x_axis), FadeIn(cat_y_axis), FadeIn(cat_x), FadeIn(cat_y), FadeIn(x), FadeIn(y), run_time = 3)
        self.play(Unwrite(text1), run_time = 1.5)
        self.wait(0.5)
        
        time = ValueTracker(0)

        bird.add_updater(bird_updater)
        cat.add_updater(cat_updater)
        cat_x_axis.add_updater(cat_x_axis_updater)
        cat_y_axis.add_updater(cat_y_axis_updater)
        cat_x.add_updater(cat_x_updater)
        cat_y.add_updater(cat_y_updater)
        x.add_updater(x_updater)
        y.add_updater(y_updater)

        self.play(time.animate.set_value(7.5), rate_func = linear, run_time = 7.5)
        #self.wait(1.5)
        cat.remove_updater(cat_updater)
        self.play(FadeOut(cat), FadeOut(bird), FadeOut(cat_x_axis), FadeOut(cat_y_axis), FadeOut(cat_x), FadeOut(cat_y), run_time = 4.5)
        self.play(Write(text2), run_time = 1.5)

        bird.remove_updater(bird_updater)
        bird.move_to(npla.c2p(traj_bird(0)[0], traj_bird(0)[1], 0))
        time = ValueTracker(0)

        cat.add_updater(cat_updater)
        self.play(FadeIn(cat), FadeIn(bird), FadeIn(cat_x_axis), FadeIn(cat_y_axis), FadeIn(cat_x), FadeIn(cat_y), run_time = 3)

        self.play(Unwrite(text2), run_time = 1.5)
        self.wait(0.5)        

        bird.add_updater(bird_updater)
        npla.add_updater(npla_updater)

        self.play(time.animate.set_value(7.5), rate_func = linear, run_time = 7.5)
        cat.remove_updater(cat_updater)
        self.play(FadeOut(cat), FadeOut(bird), FadeOut(cat_x_axis), FadeOut(cat_y_axis), FadeOut(cat_x), FadeOut(cat_y), run_time = 4.5)
        self.wait(3)

Output saved by overwring previous file at animations/moved_frame_of_reference/moved_frame_of_reference_F4.mp4.
